# 6강

스코어를 소프트맥스라는 함수에 통과시켜 확률처럼 사용하게 해준다.

> 어떤 레이블인가에대한 확률로 나온다.

In [48]:
import tensorflow as tf

In [2]:
x_data = [[1, 2, 1, 1],
          [2, 1, 3, 2],
          [3, 1, 3, 4],
          [4, 1, 5, 5],
          [1, 7, 5, 5],
          [1, 2, 5, 6],
          [1, 6, 6, 6],
          [1, 7, 7, 7]]
y_data = [[0, 0, 1],
          [0, 0, 1],
          [0, 0, 1],
          [0, 1, 0],
          [0, 1, 0],
          [0, 1, 0],
          [1, 0, 0],
          [1, 0, 0]]

In [6]:
x = tf.placeholder('float', [None, 4])
y = tf.placeholder('float', [None, 3]) # 원핫일때, 레이블의 갯수이다.
nb_classes = 3

W = tf.Variable(tf.random_normal([4, nb_classes]), name = 'weight')
b = tf.Variable(tf.random_normal([nb_classes]), name = 'bias')

In [8]:
hypothesis = tf.nn.softmax(tf.matmul(x, W)+b)

In [11]:
cost = tf.reduce_mean(-tf.reduce_sum(y * tf.log(hypothesis), axis = 1))
optimizer = tf.train.GradientDescentOptimizer(learning_rate= 0.1).minimize(cost)

Instructions for updating:
Use tf.cast instead.


In [19]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for step in range(2001):
        sess.run(optimizer, feed_dict = {x:x_data, y:y_data})
        if step % 200 == 0:
            print(step, sess.run(cost, feed_dict = {x:x_data, y:y_data}))
            print('--------------')
    # Testing & One-hot encoding
            a = sess.run(hypothesis, feed_dict={x: [[1, 11, 7, 9]]})
            print(a, sess.run(tf.argmax(a, 1)))

            print('--------------')
            b = sess.run(hypothesis, feed_dict={x: [[1, 3, 4, 3]]})
            print(b, sess.run(tf.argmax(b, 1)))

            print('--------------')
            c = sess.run(hypothesis, feed_dict={x: [[1, 1, 0, 1]]})
            print(c, sess.run(tf.argmax(c, 1)))

            print('--------------')
            all = sess.run(hypothesis, feed_dict={x: [[1, 11, 7, 9], [1, 3, 4, 3], [1, 1, 0, 1]]})
            print(all, sess.run(tf.argmax(all, 1)))

0 4.3460183
--------------
[[4.4644311e-01 1.7510966e-10 5.5355692e-01]] [2]
--------------
[[9.063399e-01 8.685963e-05 9.357320e-02]] [0]
--------------
[[0.01338154 0.01340834 0.9732101 ]] [2]
--------------
[[4.4644311e-01 1.7510966e-10 5.5355692e-01]
 [9.0633988e-01 8.6859633e-05 9.3573198e-02]
 [1.3381543e-02 1.3408344e-02 9.7321010e-01]] [2 0 2]
200 0.45783472
--------------
[[0.01859414 0.97922486 0.00218105]] [1]
--------------
[[0.63675624 0.31232616 0.0509176 ]] [0]
--------------
[[6.4569648e-04 3.9473809e-02 9.5988053e-01]] [2]
--------------
[[1.8594144e-02 9.7922486e-01 2.1810518e-03]
 [6.3675624e-01 3.1232616e-01 5.0917592e-02]
 [6.4569648e-04 3.9473809e-02 9.5988053e-01]] [1 0 2]
400 0.36584038
--------------
[[9.3054511e-03 9.9034661e-01 3.4797087e-04]] [1]
--------------
[[0.70400316 0.25445467 0.04154212]] [0]
--------------
[[8.0498576e-05 1.7740462e-02 9.8217899e-01]] [2]
--------------
[[9.3054501e-03 9.9034661e-01 3.4797087e-04]
 [7.0400316e-01 2.5445467e-01 4.15

__arg_max(데이터, (axis에서의 최강자를뽑아주세요)축)__

가장 큰 값을 골라서 인덱스를 리턴해준다.

## fancy하게 softmax를 만들어보자

In [50]:
import numpy as np

In [69]:
xy = np.loadtxt('data/data-04-zoo.csv', delimiter = ',', dtype = np.float32)
x_data = xy[:, 0:-1]
y_data = xy[:, [-1]]

# 7가지 종으로 분류

In [70]:
# 원핫으로 바꾸어준다.
nb_classes = 7

X = tf.placeholder(tf.float32, [None, 16])
Y = tf.placeholder(tf.int32, [None, 1])  # 0 ~ 6

Y_one_hot = tf.one_hot(Y, nb_classes) # 아웃풋이 n+1차원으로 나온다.
# 그래서 (?, 7)로 해줘야한다. -1은 everything
Y_one_hot = tf.reshape(Y_one_hot, [-1, nb_classes])

In [71]:
W = tf.Variable(tf.random_normal([16, nb_classes]), name='weight')
b = tf.Variable(tf.random_normal([nb_classes]), name='bias')

# softmax = exp(logits) / reduce_sum(exp(logits), dim)
logits = tf.matmul(X, W) + b
hypothesis = tf.nn.softmax(logits)

In [74]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits,
                                                                 labels=Y_one_hot))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)

# 확률을 0~6사이의 값중 하나로 만들어 낸다. 
# 가장 큰 값의 인덱스 반환.
prediction = tf.argmax(hypothesis, 1)
# 실제의 값과 예측값 비교.
correct_prediction = tf.equal(prediction, tf.argmax(Y_one_hot, 1))
# 맞는 것들 평균낸다.
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

Tensor("ArgMax_887:0", shape=(?,), dtype=int64)


In [73]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(y_data.shape)
    for step in range(2001):
        _, cost_val, acc_val = sess.run([optimizer, cost, accuracy], feed_dict={X: x_data, Y: y_data})
                                        
        if step % 100 == 0:
            print("Step: {}\tCost: {}\tAcc: {}".format(step, cost_val, acc_val))

    # Let's see if we can predict
    pred = sess.run(prediction, feed_dict={X: x_data})
    # y_data: (N,1) = flatten => (N, ) matches pred.shape. 평평하게 해준다는 거다.
    for p, y in zip(pred, y_data.flatten()):
        print("[{}] Prediction: {} True Y: {}".format(p == int(y), p, int(y)))

(101, 1)
Step: 0	Cost: 5.701645851135254	Acc: 0.0891089141368866
Step: 100	Cost: 0.7376276850700378	Acc: 0.8613861203193665
Step: 200	Cost: 0.48127636313438416	Acc: 0.8910890817642212
Step: 300	Cost: 0.35100501775741577	Acc: 0.9009901285171509
Step: 400	Cost: 0.27868667244911194	Acc: 0.9306930899620056
Step: 500	Cost: 0.23379281163215637	Acc: 0.9504950642585754
Step: 600	Cost: 0.20194265246391296	Acc: 0.9603960514068604
Step: 700	Cost: 0.1775452196598053	Acc: 0.9603960514068604
Step: 800	Cost: 0.15805691480636597	Acc: 0.9801980257034302
Step: 900	Cost: 0.1420888751745224	Acc: 0.9801980257034302
Step: 1000	Cost: 0.12877796590328217	Acc: 0.9801980257034302
Step: 1100	Cost: 0.1175382137298584	Acc: 0.9801980257034302
Step: 1200	Cost: 0.10794739425182343	Acc: 0.9900990128517151
Step: 1300	Cost: 0.09968928247690201	Acc: 0.9900990128517151
Step: 1400	Cost: 0.09252102673053741	Acc: 1.0
Step: 1500	Cost: 0.08625277876853943	Acc: 1.0
Step: 1600	Cost: 0.08073440194129944	Acc: 1.0
Step: 1700	Cost: 